In [1]:
# General Imports

import numpy as np
from numpy import random as rnd
from matplotlib import pyplot as plt
import os,sys,datetime,time,math, warnings,itertools

import pandas as pd
from contextlib import contextmanager

# Snowpark imports
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark.window import Window

c:\Users\z003mxpm\Anaconda3\lib\site-packages\snowflake\connector\options.py:96: UserWarning: You have an incompatible version of 'pyarrow' installed (12.0.1), please install a version that adheres to: 'pyarrow<8.1.0,>=8.0.0; extra == "pandas"'
  warn_incompatible_dep(


In [2]:
@contextmanager
def cwd(path):
    """
    Context manager to temporarily change the current working directory.

    This context manager changes the current working directory to the specified 'path'
    while the context is active. After the context exits, the original working directory
    is restored.

    Args:
        path (str): The path to the directory to which the current working directory should be changed.

    Usage:
        with cwd('/path/to/new/directory'):
            # Code executed within this block will have the current working directory changed.
            # After the block exits, the original working directory is restored.

    Example:
        with cwd('/home/user/documents'):
            file_list = os.listdir()  # List files in the '/home/user/documents' directory
        # 
    """
    oldpwd = os.getcwd()
    os.chdir(path)
    try: yield
    finally: os.chdir(oldpwd)

In [3]:
# Path settings
if sys.platform.startswith('win'):
    excel_directory = r'../../TF_Data/Dropbox/PhD Prep/DQ Framework - Clustering/Data Products/'

In [6]:
with cwd(excel_directory):
    df = pd.read_excel(io='Funnel Management Data Product.xlsx',sheet_name='Data Assets per Data Product')
    qf = pd.read_excel(io='DQ Evaluation.xlsx',sheet_name='Sheet1')

In [14]:
ddf = df[['DP Table','Asset Name']].drop_duplicates().merge(
    qf.drop(columns=['Table Rowcount Total']),left_on='Asset Name',right_on='Source Table',how='left'
)

ddf[~ddf['Source Table'].isna()]

,DP Table,Asset Name,Source Table,DQ Flag Name,DQ Dimension,Quality Percentage [%]
1,All_Status (Last-NextFY),Accounts,Accounts,Potential Test Account (Flag),Correctness,0.998663
2,All_Status (Last-NextFY),OLIs,OLIs,Quote Item Reference not found (Flag),Completeness,0.931514
3,All_Status (Last-NextFY),OLIs,OLIs,Multiple Quotes attached (Flag),Uniquness,0.926086
4,All_Status (Last-NextFY),OLIs,OLIs,DQ Issue In-Vitro Opportunity related (Flag),Correctness,0.672110
5,All_Status (Last-NextFY),OLIs,OLIs,Primary Winning Vendor Unknown (Flag),Completeness,0.996965
...,...,...,...,...,...,...
322,Win_Loss (Last-CurrentFY),OLIs,OLIs,Installation Date Wrong (Flag),Correctness,0.968103
323,Win_Loss (Last-CurrentFY),OLIs,OLIs,DQ Issue Quote-related (Flag),Correctness,0.841020
324,Win_Loss (Last-CurrentFY),OLIs,OLIs,Delivery Date Wrong (Flag),Correctness,0.996321
325,Win_Loss (Last-CurrentFY),OLIs,OLIs,DQ Issue In-Vitro Funnel relevant (Flag),Correctness,0.787255


In [122]:
# Connection Parameters
connection_parameters = {
    'account':'shsitdl.west-europe.azure',
    'user':'jan-lucas.deinhard@siemens-healthineers.com',
    'authenticator':'externalbrowser',
    'role':'FR_CRMCLOUD_DEV',
    'database':'MARTLAYER',
    'schema':'INFORMATION_SCHEMA',
    'warehouse':'W_CRMCLOUD_P'
}

# Establish Connection
session = Session.builder.configs(connection_parameters).create()

cC = session.table("COLUMNS")
C = pd.DataFrame(cC.collect())

session.close()

# Merge all columns into the DP-relevant tables
dq = df.merge(
    C[['TABLE_CATALOG','TABLE_SCHEMA','TABLE_NAME','COLUMN_NAME']].drop_duplicates(),
    on=['TABLE_CATALOG','TABLE_SCHEMA','TABLE_NAME'],
    how='left'
)